In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from PIL import Image

import torch
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn

In [3]:
def evaluate_image(path_file):
    # Load the ResNet50 model 
    model = models.resnet50(weights=None)

    # Replace the last layer with your desired number of classes (e.g., 13)
    num_classes = 13
    model.fc = nn.Linear(2048, num_classes)

    # Replace the model weights with your own
    model.load_state_dict(torch.load(os.path.join('.','models','ft_resnet50.pt'), map_location=torch.device('cpu')))

    mean = [0.56725096, 0.46618392, 0.36616178]
    std = [0.28435631, 0.28744022, 0.29725166]
    class_names = [
        'bhaji', 'chapati', 'githeri', 
        'kachumbari', 'kukuchoma', 'mandazi', 
        'masalachips', 'matoke', 'mukimo', 
        'nyamachoma', 'pilau', 'sukumawiki', 
        'ugali'
    ]

    # Prepare the transformation for the input image
    # Adjust to the size and format of your images
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),  
    ])

    img = Image.open(path_file)

    # Apply the transformation
    img_tensor = transform(img)
    
    # Add another dimension for batch (because the model expects batch)
    img_tensor = img_tensor.unsqueeze(0)
    
    # Switch the model to eval mode
    model.eval()
    
    # Turn off gradient calculation (for inference)
    with torch.no_grad():
        # Forecast
        outputs = model(img_tensor)
    
    # Get the index of the class with the highest probability
    _, predicted = torch.max(outputs, 1)

    results_class_name = class_names[predicted]

    return results_class_name

In [4]:
test_images = pd.read_csv(os.path.join('.','data','test.csv'))
test_images.head(10)

,id
0,9156739011499789258
1,2049465964503133373
2,6446998501027132988
3,4194396063119815321
4,9018117998187006009
5,6246759883907852128
6,16478122708528316044
7,14045745760440690312
8,7872954221890963019
9,4868486697531317477


In [5]:
def create_path(id):
    base_path = os.path.join('.','data','images','images')
    return os.path.join(base_path, str(id) + ".jpg")

In [6]:
# Suppose you have a DataFrame df with a column 'ID'
test_images['path_file'] = test_images['id'].apply(create_path)
test_images.sample(10)

,id,path_file
808,17177855810863632504,.\data\images\images\17177855810863632504.jpg
480,15265065584326275962,.\data\images\images\15265065584326275962.jpg
170,8836295853288823747,.\data\images\images\8836295853288823747.jpg
1541,2200281231231374883,.\data\images\images\2200281231231374883.jpg
1478,16856724721935104840,.\data\images\images\16856724721935104840.jpg
370,9483664308571127021,.\data\images\images\9483664308571127021.jpg
1076,13000400453951407825,.\data\images\images\13000400453951407825.jpg
667,15662555471234479295,.\data\images\images\15662555471234479295.jpg
903,2906555109236171415,.\data\images\images\2906555109236171415.jpg
25,14524516184482112107,.\data\images\images\14524516184482112107.jpg


In [7]:
len(test_images)

1638

In [8]:
target_class = pd.read_csv(os.path.join('.','data','sample_submission.csv'))
target_class.head(10)

,id,class
0,9156739011499789258,pilau
1,2049465964503133373,nyamachoma
2,6446998501027132988,masalachips
3,4194396063119815321,nyamachoma
4,9018117998187006009,mandazi
5,6246759883907852128,ugali
6,16478122708528316044,sukumawiki
7,14045745760440690312,masalachips
8,7872954221890963019,ugali
9,4868486697531317477,mandazi


In [9]:
len(target_class)

1638

In [10]:
# Spojení tabulek
merged_df = pd.merge(test_images, target_class, on='id', how='left')
merged_df = merged_df.rename(columns={'class': 'target'})
merged_df.sample(10)

,id,path_file,target
1581,13250687501080686233,.\data\images\images\13250687501080686233.jpg,nyamachoma
1204,31710090846619110,.\data\images\images\31710090846619110.jpg,nyamachoma
966,12762109995665584616,.\data\images\images\12762109995665584616.jpg,kukuchoma
507,10339445144891593406,.\data\images\images\10339445144891593406.jpg,mandazi
314,4481342611008844605,.\data\images\images\4481342611008844605.jpg,kukuchoma
188,15851485436682147775,.\data\images\images\15851485436682147775.jpg,pilau
1406,15975300486287559149,.\data\images\images\15975300486287559149.jpg,kachumbari
1449,8818144310971459792,.\data\images\images\8818144310971459792.jpg,matoke
1436,1853546307740129133,.\data\images\images\1853546307740129133.jpg,matoke
458,7503918163130415845,.\data\images\images\7503918163130415845.jpg,kukuchoma


In [11]:
# Check if any element in the table is NaN (Not a Number)
if merged_df.isna().values.any():
    print("The table contains empty values.")
else:
    print("The table does not contain empty values.")

The table does not contain empty values.


In [12]:
len(merged_df)

1638

In [13]:
%%time
# Suppose you have a DataFrame df with a column 'class' (predicted results)
merged_df['class'] = merged_df['path_file'].apply(evaluate_image)
merged_df.sample(10)

C:\Users\kozl\AppData\Local\Temp\ipykernel_11664\2668646616.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join('.','models','

CPU times: total: 5min 58s
Wall time: 10min 53s


,id,path_file,target,class
299,5377424266929477918,.\data\images\images\5377424266929477918.jpg,mandazi,ugali
554,7445824523945345833,.\data\images\images\7445824523945345833.jpg,pilau,chapati
680,15977602221596961242,.\data\images\images\15977602221596961242.jpg,kukuchoma,ugali
1004,6320387480973366855,.\data\images\images\6320387480973366855.jpg,nyamachoma,nyamachoma
804,12816264944833013721,.\data\images\images\12816264944833013721.jpg,kachumbari,matoke
985,14784080218898399737,.\data\images\images\14784080218898399737.jpg,chapati,mandazi
822,6131236831391510211,.\data\images\images\6131236831391510211.jpg,mukimo,githeri
235,8213365026002176366,.\data\images\images\8213365026002176366.jpg,mandazi,chapati
437,7454000247671709039,.\data\images\images\7454000247671709039.jpg,matoke,chapati
1621,17564317916912636892,.\data\images\images\17564317916912636892.jpg,ugali,githeri


In [14]:
# Check if any element in the table is NaN (Not a Number)
if merged_df.isna().values.any():
    print("The table contains empty values.")
else:
    print("The table does not contain empty values.")

The table does not contain empty values.


In [15]:
len(merged_df)

1638

In [17]:
# export results
export_results = merged_df.drop(['path_file', 'target'], axis=1)
export_results.head(10)

,id,class
0,9156739011499789258,nyamachoma
1,2049465964503133373,kachumbari
2,6446998501027132988,nyamachoma
3,4194396063119815321,ugali
4,9018117998187006009,bhaji
5,6246759883907852128,masalachips
6,16478122708528316044,nyamachoma
7,14045745760440690312,kachumbari
8,7872954221890963019,matoke
9,4868486697531317477,chapati


In [18]:
# Save 
export_results.to_csv(os.path.join('.','results', 'ft_resnet50_evalution.csv'), index=False)

In [19]:
mask_res_true = merged_df['target'] == merged_df['class']
mask_res_false = merged_df['target'] != merged_df['class']

In [20]:
true_results = merged_df[mask_res_true]
print(f'Number of true prediction {len(true_results)}')
true_results.head(5)

Number of true prediction 131


,id,path_file,target,class
74,12831757841808404416,.\data\images\images\12831757841808404416.jpg,ugali,ugali
86,16903252693459438162,.\data\images\images\16903252693459438162.jpg,chapati,chapati
124,3329821741742186938,.\data\images\images\3329821741742186938.jpg,bhaji,bhaji
138,15911324942732885782,.\data\images\images\15911324942732885782.jpg,kachumbari,kachumbari
145,8510036279344254906,.\data\images\images\8510036279344254906.jpg,mandazi,mandazi


In [21]:
false_results = merged_df[mask_res_false]
print(f'Number of false prediction {len(false_results)}')
false_results.head(5)

Number of false prediction 1507


,id,path_file,target,class
0,9156739011499789258,.\data\images\images\9156739011499789258.jpg,pilau,nyamachoma
1,2049465964503133373,.\data\images\images\2049465964503133373.jpg,nyamachoma,kachumbari
2,6446998501027132988,.\data\images\images\6446998501027132988.jpg,masalachips,nyamachoma
3,4194396063119815321,.\data\images\images\4194396063119815321.jpg,nyamachoma,ugali
4,9018117998187006009,.\data\images\images\9018117998187006009.jpg,mandazi,bhaji


In [24]:
print(f'Accuracy of model is {(len(true_results)/(len(true_results)+len(false_results)))*100}%')

Accuracy of model is 7.997557997557997%
